# Prep Data to use in Tableau for Stacked Area chart and Sankey diagram

## Summary of Cells
- Imports + Loggings Configuration + Set Constants
- Data Acquisition: Handles downloading and loading the dataset.
- Data Preprocessing: Handles initial cleaning and merging.
- Data Completion: Deals with filling missing entries and creating a full dataset.
- Output: Saves the results to a file.
- Orchestrator: Executes the workflow in order.

## Imports + Logging + Constants

In [1]:
# Required Imports
import os
import logging
from typing import Tuple
import pandas as pd
import kagglehub

# Configure Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configuration Constants
DATASET_PATH = "rohanrao/formula-1-world-championship-1950-2020"
OUTPUT_CSV = 'drivers_constr.csv'
YEAR_START = 1999
YEAR_END = 2023
DEFAULT_RANK = 42  # Arbitrary number to indicate absence

## Data Acquisition
- Download files
- Read files

In [2]:
def download_dataset(dataset: str) -> str:
    """
    Downloads the dataset using kagglehub and returns the folder path.
    """
    try:
        folder = kagglehub.dataset_download(dataset)
        logging.info(f"Dataset downloaded successfully: {dataset}")
        return folder
    except Exception as e:
        logging.error(f"Failed to download dataset {dataset}: {e}")
        raise

def read_csv_file(folder_path: str, filename: str) -> pd.DataFrame:
    """
    Reads a CSV file from the specified folder path.
    """
    path = os.path.join(folder_path, filename)
    try:
        df = pd.read_csv(path)
        logging.info(f"Loaded {filename} successfully.")
        return df
    except FileNotFoundError:
        logging.error(f"File not found: {path}")
        raise
    except pd.errors.ParserError as e:
        logging.error(f"Error parsing {path}: {e}")
        raise

## Data Preprocessing

In [3]:
def preprocess_data(results: pd.DataFrame, races: pd.DataFrame, driver_standings: pd.DataFrame) -> pd.DataFrame:
    """
    Processes and merges the datasets to prepare for analysis.
    """
    # Select relevant columns
    logging.info("Selecting relevant columns from results.")
    year_standings = results[['raceId', 'driverId', 'constructorId']].copy()

    # Merge with races to get year
    logging.info("Merging with races data to include year information.")
    year_standings = pd.merge(
        year_standings,
        races[['raceId', 'year']],
        on='raceId',
        how='left'
    )

    # Merge with driver standings to get points
    logging.info("Merging with driver standings to include points.")
    year_standings = pd.merge(
        year_standings,
        driver_standings[['raceId', 'driverId', 'points']],
        on=['raceId', 'driverId'],
        how='left'
    )

    # Filter years
    logging.info(f"Filtering data for years between {YEAR_START} and {YEAR_END}.")
    year_standings = year_standings[
        (year_standings["year"] >= YEAR_START) & (year_standings["year"] <= YEAR_END)
    ]

    # Aggregate to get maximum points per driver-constructor-year
    logging.info("Aggregating to get maximum points per driver-constructor-year.")
    year_standings = year_standings.groupby(["driverId", "constructorId", "year"], as_index=False)['points'].max()

    # Drop entries with no points
    logging.info("Dropping entries where drivers haven't scored points.")
    year_standings = year_standings.dropna(subset=['points'])
    year_standings = year_standings[year_standings['points'] > 0]

    return year_standings

def add_constructor_attributes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds constructor_points, constructor_points(%), and rank to the dataframe.
    """
    logging.info("Adding constructor_points attribute.")
    df['constructor_points'] = df.groupby(['year', 'constructorId'])['points'].transform('sum')

    logging.info("Calculating constructor_points(%) attribute.")
    yearly_points_sum = df.groupby('year')['points'].transform('sum')
    df['constructor_points(%)'] = (df['constructor_points'] / yearly_points_sum * 100).round().astype(int)

    # Convert relevant columns to integers
    logging.info("Converting points-related columns to integers.")
    df['points'] = df['points'].astype(int)
    df['constructor_points'] = df['constructor_points'].astype(int)

    # Add rank based on constructor_points
    logging.info("Calculating rank based on constructor_points.")
    df['rank'] = df.groupby('year')['constructor_points'].rank(ascending=False, method='dense').astype(int)

    return df

## Data Completion

In [4]:
def add_buffer_entries(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds buffer entries for constructors in years they were not active.
    """
    logging.info("Adding buffer entries for constructors not present in certain years.")
    constructor_ids = df['constructorId'].unique()
    years = range(YEAR_START, YEAR_END + 1)

    # Create complete grid
    logging.info("Creating a complete grid of all year-constructor combinations.")
    full_index = pd.MultiIndex.from_product(
        [years, constructor_ids],
        names=['year', 'constructorId']
    )
    full_year_con = pd.DataFrame(index=full_index).reset_index()

    # Merge with existing data
    logging.info("Merging complete grid with existing standings data.")
    full_year_con = full_year_con.merge(
        df,
        on=['year', 'constructorId'],
        how='left'
    )

    # Fill missing values
    logging.info("Filling missing values with defaults.")
    full_year_con['points'] = full_year_con['points'].fillna(0).astype(int)
    full_year_con['driverId'] = full_year_con['driverId'].fillna(0).astype(int)
    full_year_con['rank'] = full_year_con['rank'].fillna(DEFAULT_RANK).astype(int)
    full_year_con['constructor_points'] = full_year_con['constructor_points'].fillna(0).astype(int)
    full_year_con['constructor_points(%)'] = full_year_con['constructor_points(%)'].fillna(0).astype(int)

    # Sort for readability
    logging.info("Sorting the DataFrame for better readability.")
    full_year_con = full_year_con.sort_values(by=['constructorId', 'year']).reset_index(drop=True)

    return full_year_con

## Create Output .csv

In [5]:

def save_to_csv(df: pd.DataFrame, filename: str):
    """
    Saves the dataframe to a CSV file.
    """
    try:
        df.to_csv(filename, index=False)
        logging.info(f"Data successfully saved to {filename}.")
    except Exception as e:
        logging.error(f"Failed to save data to {filename}: {e}")
        raise

## Main Orchestrator

In [6]:
def main():
    """
    Main function to orchestrate data processing.
    """
    logging.info("Starting data processing pipeline.")

    # Step 1: Download Dataset
    folder_path = download_dataset(DATASET_PATH)

    # Step 2: Read CSV Files
    results = read_csv_file(folder_path, "results.csv")
    races = read_csv_file(folder_path, "races.csv")
    driver_standings = read_csv_file(folder_path, "driver_standings.csv")

    # Step 3: Preprocess Data
    year_standings = preprocess_data(results, races, driver_standings)

    # Step 4: Add Constructor Attributes
    year_standings = add_constructor_attributes(year_standings)

    # Step 5: Add Buffer Entries
    full_year_con = add_buffer_entries(year_standings)

    # Step 6: Save to CSV
    save_to_csv(full_year_con, OUTPUT_CSV)

    # Step 7: Display Data Information
    logging.info("Displaying DataFrame information:")
    full_year_con.info()
    logging.info("Displaying first few rows of the DataFrame:")
    display(full_year_con.head())

if __name__ == "__main__":
    main()

2025-01-02 11:35:56,225 - INFO - Starting data processing pipeline.


2025-01-02 11:35:56,882 - INFO - Dataset downloaded successfully: rohanrao/formula-1-world-championship-1950-2020
2025-01-02 11:35:56,930 - INFO - Loaded results.csv successfully.
2025-01-02 11:35:56,935 - INFO - Loaded races.csv successfully.
2025-01-02 11:35:56,955 - INFO - Loaded driver_standings.csv successfully.
2025-01-02 11:35:56,956 - INFO - Selecting relevant columns from results.
2025-01-02 11:35:56,958 - INFO - Merging with races data to include year information.
2025-01-02 11:35:56,963 - INFO - Merging with driver standings to include points.
2025-01-02 11:35:56,974 - INFO - Filtering data for years between 1999 and 2023.
2025-01-02 11:35:56,976 - INFO - Aggregating to get maximum points per driver-constructor-year.
2025-01-02 11:35:56,980 - INFO - Dropping entries where drivers haven't scored points.
2025-01-02 11:35:56,982 - INFO - Adding constructor_points attribute.
2025-01-02 11:35:56,984 - INFO - Calculating constructor_points(%) attribute.
2025-01-02 11:35:56,985 - I

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   year                   1046 non-null   int64
 1   constructorId          1046 non-null   int64
 2   driverId               1046 non-null   int64
 3   points                 1046 non-null   int64
 4   constructor_points     1046 non-null   int64
 5   constructor_points(%)  1046 non-null   int64
 6   rank                   1046 non-null   int64
dtypes: int64(7)
memory usage: 57.3 KB


,year,constructorId,driverId,points,constructor_points,constructor_points(%),rank
0,1999,1,14,48,124,30,2
1,1999,1,57,76,124,30,2
2,2000,1,14,73,162,37,2
3,2000,1,57,89,162,37,2
4,2001,1,14,65,102,23,2
